# Evaluation of experiments - E1 COGNITION
fistly define the UUID of the experiment to use

In [1]:
experimentId="352338b0-f723-421f-a69e-e2afb40205ed"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load cognition

In [3]:
samplerate=200  #Should rethink this

In [11]:
annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath)

In [12]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

2785

In [13]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Load experiment results

In [14]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [15]:
experimentModels.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001,16b9b403-5b57-4f9b-b9b4-dd56240acf77,0.008985
1,0001,E1,[0001],"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002,07e6bbaa-2aa2-4df4-8e5f-28bc716dc817,0.005890
2,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003,53f1c280-5951-4042-852e-00443c71b12e,0.005867
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004,e8ce0b00-6a48-4362-bc01-501a4996914c,0.008876
4,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005,90ab0dfb-22de-4127-bcbc-ea6d81cf7540,0.005867


In [16]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Optimal hyperparameter estimation with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [17]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0008]","[0009, 0004]",0001,16b9b403-5b57-4f9b-b9b4-dd56240acf77,0.008985
1,0001,E1,[0001],"[0004, 0005, 0006, 0007, 0008, 0009]","[0001, 0003]",0002,07e6bbaa-2aa2-4df4-8e5f-28bc716dc817,0.005890
2,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0005, 0006]",0003,53f1c280-5951-4042-852e-00443c71b12e,0.005867
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009]","[0005, 0008]",0004,e8ce0b00-6a48-4362-bc01-501a4996914c,0.008876
4,0001,E1,[0001],"[0001, 0002, 0004, 0007, 0008, 0009]","[0003, 0006]",0005,90ab0dfb-22de-4127-bcbc-ea6d81cf7540,0.005867
5,0001,E1,[0001],"[0003, 0004, 0005, 0007, 0008, 0009]","[0001, 0002]",0006,fd90c505-fd19-4b2a-bbb7-a88529c36d7c,0.009365
6,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0008]","[0006, 0009]",0007,5278edde-a3c5-4161-9613-f4be8bd14fc0,0.009711
7,0001,E1,[0001],"[0001, 0002, 0003, 0004, 0005, 0006]","[0009, 0007]",0008,81a44421-f8a4-48c9-9dba-9aa04ab97659,0.010361
8,0001,E1,[0001],"[0001, 0003, 0004, 0005, 0006, 0008]","[0007, 0002]",0009,f1d48762-3da9-4fe4-920f-943298435e53,0.009431


In [18]:
auxThres=pd.DataFrame({
    'hyperThres':[0.3,0.4,0.5,0.6]
})
auxDepth=pd.DataFrame({
    'hyperDepth':[20,30,40,50,60]
})
auxDuration=pd.DataFrame({
    'hyperDuration':[0.3]
})
auxClose=pd.DataFrame({
    'hyperClose':[0.25]
})
auxConfidence=pd.DataFrame({
    'hyperConfidence':[0]
})

hyperParams=pd.merge(auxThres,auxDepth,how='cross')
hyperParams=pd.merge(hyperParams,auxDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxClose,how='cross')
hyperParams=pd.merge(hyperParams,auxConfidence,how='cross')

hyperParams

,hyperThres,hyperDepth,hyperDuration,hyperClose,hyperConfidence
0,0.3,20,0.3,0.25,0
1,0.3,30,0.3,0.25,0
2,0.3,40,0.3,0.25,0
3,0.3,50,0.3,0.25,0
4,0.3,60,0.3,0.25,0
5,0.4,20,0.3,0.25,0
6,0.4,30,0.3,0.25,0
7,0.4,40,0.3,0.25,0
8,0.4,50,0.3,0.25,0
9,0.4,60,0.3,0.25,0


In [19]:
def getCoordAndLenList(labelVector):
    objects=ndi.find_objects(ndi.label(labelVector)[0])
    coords=list(map(lambda x: (x[0].start,x[0].stop,x[0].stop-x[0].start),objects))
    return coords
    
def rawAndProcessedToConfidenceAnnotations(raw,processed,samplerate):
    #calculate the number of true raw outputs in each detection
    labels=ndi.label(processed)
    actualPredicted=np.zeros((labels[1],))
    for i in np.arange(0,labels[1])+1:
        actualPredicted[i-1]=np.sum(raw*(labels[0]==i))
    #calculate the confidences
    outCoords=getCoordAndLenList(processed)
    confidences=[]
    for coords,actual in zip(outCoords,actualPredicted):
        confidences.append(actual/coords[2])
    #create a data frame
    confidenceDf=pd.DataFrame({
        'startInd':[x[0] for x in outCoords],
        'stopInd':[x[1] for x in outCoords],
        'duration':[x[2]/samplerate for x in outCoords],
        'confidence':confidences
    })
    return confidenceDf

In [20]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [21]:
hyperExperimentModels=[]
hyperMeanF1=[]
hyperStdF1=[]

for ind_hyper,row_hyper in hyperParams.iterrows():
    print(ind_hyper)
    thisExperimentModels=experimentModels.copy()
    hyperThres=row_hyper.hyperThres
    hyperDepth=int(row_hyper.hyperDepth)
    hyperDuration=row_hyper.hyperDuration
    hyperClose=row_hyper.hyperClose
    """ hyperConfidence=row_hyper.hyperConfidence """
    
    meanF1=[]
    meanPrecision=[]
    meanRecall=[]

    stdF1=[]
    stdPrecision=[]
    stdRecall=[]

    for ind,row in thisExperimentModels.iterrows():
        #load model
        model=loadBooster(row.modelId,experimentId,datapath)
        #initialise lists
        rawF1s=[]
        rawPrecisions=[]
        rawRecalls=[]

        f1s=[]
        precisions=[]
        recalls=[]

        #iterate validation subjects
        for valSubjectId in row.val:
            #Define annotations criterium
            usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
            #Load features and labels
            valFeatures=loadFeatureMatrix([valSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
            valLabels=loadLabelsVector([valSubjectId],usedAnnotations,signalsMetadata,samplerate)
            #Predict
            valDMatrix=xgb.DMatrix(data=valFeatures)
            probabilities=model.predict(valDMatrix,iteration_range=(0,hyperDepth))
            raw=probabilities>=hyperThres
            #Processed labels
            processed=labelingProcess(raw,hyperClose,hyperDuration,samplerate)
            """ #Detections with confidence
            detections=rawAndProcessedToConfidenceAnnotations(raw,processed,samplerate)
            #Confidence threshold
            detections=detections[detections.confidence>hyperConfidence].reset_index(drop=True) """
            gtAnnotations=labelVectorToAnnotations(valLabels,samplerate)
            detections=labelVectorToAnnotations(processed,samplerate)
            #Metrics
            f,r,p=annotationPairToMetrics(gtAnnotations,detections)
            
            #Metric appends
            f1s.append(f)
            precisions.append(p)
            recalls.append(r)

        #statistics of the metrics over the subjects of the validation set
        meanF1.append(np.mean(f1s))
        meanPrecision.append(np.mean(precisions))
        meanRecall.append(np.mean(recalls))

        stdF1.append(np.std(f1s))
        stdPrecision.append(np.std(precisions))
        stdRecall.append(np.std(recalls))
        
    thisExperimentModels['meanF1']=meanF1
    thisExperimentModels['meanPrecision']=meanPrecision
    thisExperimentModels['meanRecall']=meanRecall
    
    thisExperimentModels['stdF1']=stdF1
    thisExperimentModels['stdPrecision']=stdPrecision
    thisExperimentModels['stdRecall']=stdRecall

    hyperExperimentModels.append(thisExperimentModels)
    hyperMeanF1.append(np.mean(thisExperimentModels['meanF1']))
    hyperStdF1.append(np.std(thisExperimentModels['meanF1']))
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [22]:
hyperParams['meanMeanF1']=hyperMeanF1
hyperParams['stdMeanF1']=hyperStdF1

In [23]:
optimalInd=hyperParams[hyperParams.meanMeanF1==np.max(hyperParams.meanMeanF1)].index[0]
print("maximal mean score at:")
optimal=hyperParams.iloc[optimalInd]
optimal

maximal mean score at:


hyperThres          0.400000
hyperDepth         30.000000
hyperDuration       0.300000
hyperClose          0.250000
hyperConfidence     0.000000
meanMeanF1          0.543101
stdMeanF1           0.127187
Name: 6, dtype: float64

## Graphical representation of the optimal point

In [24]:
aux=hyperParams[(hyperParams.hyperDepth==optimal.hyperDepth)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperThres',y='meanMeanF1',error_y='stdMeanF1')

In [25]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperDepth',y='meanMeanF1',error_y='stdMeanF1')

In [26]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperDuration',y='meanMeanF1',error_y='stdMeanF1')

In [27]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperDuration==optimal.hyperDuration)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperClose',y='meanMeanF1',error_y='stdMeanF1')

In [37]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperDuration==optimal.hyperDuration)&(hyperParams.hyperClose==optimal.hyperClose)].reset_index(drop=True)
px.scatter(aux,x='hyperConfidence',y='meanMeanF1',error_y='stdMeanF1')